In [ ]:
!nvidia-smi

Thu Oct 12 06:44:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
  Obtaining dependency information for transformers>=4.31.0 from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 1.9 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.15,>=0.14 from https://files.pythonhosted.org/packages/a7/7b/c1f643eb086b6c5c33eef0c3752e37624bd23e4cbc9f1332748f1c6252d1/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
Using cached transformers-4.34.0-py3-none-any.whl (7.7 MB)
Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attemptin

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq  chromadb==0.3.26 --progress-bar off
!pip install -qqq  sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq  autogptq==0.2.2 --progress-bar off
!pip install -qqq  einops==0.6.1 --progress-bar off
!pip install -qqq  unstructured==0.8.0 --progress-bar off
!pip install -qqq  transformers==4.30.2 --progress-bar off

ERROR: Could not find a version that satisfies the requirement autogptq==0.2.2 (from versions: none)
ERROR: No matching distribution found for autogptq==0.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-gptq 0.4.2+cu118 requires transformers>=4.31.0, but you have transformers 4.30.2 which is incompatible.


In [ ]:
from pathlib import Path
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

In [ ]:
questions_dir = Path("skyscanner")
questions_dir.mkdir(exist_ok=True, parents=True)
def write_file(question, answer, file_path):
    text = f"""
Q: {question}
A: {answer}
""".strip()
    with Path(questions_dir / file_path).open("w") as text_file:
        text_file.write(text)

In [ ]:
write_file(
    question="Is there a database with material properties in Altair EDEM?",
    answer="""
    Yes, the Generic EDEM Material Model (GEMM) database contains thousands of material models representing a variety of materials like rocks, soils and ores.The GEMM Database is fully integrated into EDEM which means users can access a range of materials instantly and can fully set up materials for a simulation without being a DEM expert.
    The GEMM database is now built into EDEM and includes even more material models!
    In addition users have access to:
    Soils starter pack: 8 models of soils including different range of compressibility and stickiness. These models make use of different physics models inbuilt in EDEM to model a range of soils from gravels to soft compressible soils.
    Powder starter pack:   9 example material models focusing on small particle sizes and representing a range of powder-materials with different flow properties and compressibility.
    """.strip(),
        file_path="question_1.txt",
)

write_file(
    question="What's new in Altair SLC Version 4.4",
    answer="""
    Altair SLC version 4.4.1 onwards is available for purchase under Altair Units using Altair Licensing.

The Altair SLC 4.4 GA release contains new features and enhancements, and maintenance updates to existing functionality. Here are just a few headline items.

SAS Language Support
Support for the ODS POWERPOINT output destination.
Both the ODSLIST and ODSTEXT procedures now support the DATA option of the procedure statement, the CELLSTYLE statement and the TRANSLATE statement.
The IMPORT procedure now supports the STARTCOL, ENDCOL, STARTROW, and ENDROW statements.
The HTTP procedure now supports the DEBUG statement.
The SGPANEL procedure now supports the TEXT statement.
The SGPLOT procedure now supports the TEXT, XAXISTABLE, and YAXISTABLE statements.
The LOGISTIC procedure now supports the UNITS statement.
Support for the MCMC procedure.
Support for the MDC procedure.
Support for the OPTLP procedure.
Support for the ORTHOREG procedure.
Support for the VARMAX procedure.
Bulk insert functionality can now be used with ODBC connections to a Google BigQuery database.
Connections to a Snowflake database can now use Key Pair Authentication and federated authentication.

Workflow
A JSON Import block has been added to enable datasets in JSON-formatted files to be imported into a Workflow.
A Parameter Import block has been added to enable parameters to be imported into a Workflow as a dataset.
A Deduplicate block has been added to enable duplicate observations to be removed from an input dataset.
A Text Transform block has been added to enable text variables in an input dataset to be modified.
The Hub group now includes:
A Program Inputs block that enables Workflow parameters to be used in an executable deployment services program.
A Program Results block that provides the results of an executable deployment services program to a Workflow.
The Hub Program block is superseded by the Program Inputs and Program Results for defining input variables to, and results from, a deployment services program.
The Database Explorer view can now be used to connect to Google BigQuery, Teradata, and Hadoop database servers.
Variables from a dataset can now be grouped together for faster selection of the same multiple variables in Workflow blocks.
    """.strip(),
        file_path="question_1.txt",
)

Model

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
DEVICE

'cuda:0'

In [ ]:
# model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
# model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# model = AutoGPTQForCausalLM.from_quantized(
#     model_name_or_path,
#     model_basename=model_basename,
#     use_safetensors=True,
#     trust_remote_code=True,
#     device=DEVICE
# )

model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE,
        use_triton=use_triton,
        quantize_config=None)

generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [ ]:
question = (
    "Which programming language is more suitable for a beginner: Python or Javascript?"
)

prompt = f"""
### Instruction: {question}
### Response:
""".strip()

In [ ]:
print(prompt)

### Instruction: Which programming language is more suitable for a beginner: Python or Javascript?
### Response:


In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

CPU times: user 3.14 s, sys: 235 ms, total: 3.37 s
Wall time: 3.5 s


In [ ]:
print(tokenizer.decode(output[0]))

<s> ### Instruction: Which programming language is more suitable for a beginner: Python or Javascript?
### Response:Both Python and Javascript are suitable for beginners, but Python is generally considered easier to learn and has a more straightforward syntax. Javascript is more commonly used for web development and can be more complex due to its interaction with HTML and CSS.</s>


In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [ ]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Peg

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
response = llm(prompt)

Python is generally considered to be more suitable for beginners due to its readability and simplicity compared to JavaScript.


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": DEVICE},
)

In [ ]:
loader = DirectoryLoader("./skyscanner/", glob="**/*txt")
documents = loader.load()
len(documents)

1

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
texts[0]

Document(page_content="Q: What's new in Altair SLC Version 4.4 A: Altair SLC version 4.4.1 onwards is available for purchase under Altair Units using Altair Licensing.\n\nThe Altair SLC 4.4 GA release contains new features and enhancements, and maintenance updates to existing functionality. Here are just a few headline items.", metadata={'source': 'skyscanner/question_1.txt'})

In [ ]:
db = Chroma.from_documents(texts, embeddings)

In [ ]:
db.similarity_search("flight search")

[Document(page_content="Q: What's new in Altair SLC Version 4.4 A: Altair SLC version 4.4.1 onwards is available for purchase under Altair Units using Altair Licensing.\n\nThe Altair SLC 4.4 GA release contains new features and enhancements, and maintenance updates to existing functionality. Here are just a few headline items.", metadata={'source': 'skyscanner/question_1.txt'}),
 Document(page_content='SAS Language Support Support for the ODS POWERPOINT output destination. Both the ODSLIST and ODSTEXT procedures now support the DATA option of the procedure statement, the CELLSTYLE statement and the TRANSLATE statement. The IMPORT procedure now supports the STARTCOL, ENDCOL, STARTROW, and ENDROW statements. The HTTP procedure now supports the DEBUG statement. The SGPANEL procedure now supports the TEXT statement. The SGPLOT procedure now supports the TEXT, XAXISTABLE, and YAXISTABLE statements. The LOGISTIC procedure now supports the UNITS statement. Support for the MCMC procedure. Supp

Conversational Chain

In [ ]:
template = """
### Instruction: You're a travelling support agent that is talking to a customer. Use only the chat history and the following information
{context}
to answer ina helpful manner to the question. If you don't know the answer - say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

In [ ]:
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"], template=template
)

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="answer",
    return_messages=True,
)

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=True,
)

In [ ]:
question = "How flight search works?"
answer = chain(question)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer. Use only the chat history and the following information
Workflow A JSON Import block has been added to enable datasets in JSON-formatted files to be imported into a Workflow. A Parameter Import block has been added to enable parameters to be imported into a Workflow as a dataset. A Deduplicate block has been added to enable duplicate observations to be removed from an input dataset. A Text Transform block has been added to enable text variables in an input dataset to be modified. The Hub group now includes: A Program Inputs block that enables Workflow parameters to be used in an executable deployment services program. A Program Results block that provides the results of an executable deployment services program to a Workflow. The Hub Program block is superseded by the Program Inputs and Program Results for defining input variab

In [ ]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [ ]:
answer["source_documents"]

[Document(page_content='Workflow A JSON Import block has been added to enable datasets in JSON-formatted files to be imported into a Workflow. A Parameter Import block has been added to enable parameters to be imported into a Workflow as a dataset. A Deduplicate block has been added to enable duplicate observations to be removed from an input dataset. A Text Transform block has been added to enable text variables in an input dataset to be modified. The Hub group now includes: A Program Inputs block that enables Workflow parameters to be used in an executable deployment services program. A Program Results block that provides the results of an executable deployment services program to a Workflow. The Hub Program block is superseded by the Program Inputs and Program Results for defining input variables to, and results from, a deployment services program. The Database Explorer view can now be used to connect to Google BigQuery, Teradata, and Hadoop database servers. Variables from a datase

In [ ]:
question = "I bought flight tickets, but I can't find any confirmation. Where is it?"
response = chain(question)



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How flight search works?
Assistant: Flight search engines work by scanning through their databases of available flights based on specific criteria provided by users such as departure airport, arrival airport, travel dates, number of passengers, class of service, etc. They then display a list of matching flights along with relevant details like flight duration, layover time, stopovers, cost, and availability. Some popular flight search engines also allow users to filter their searches according to preferences like nonstop flights or those with layovers, preferred airlines, price range, and amenities.
Follow Up Input: I bought flight tickets, but I can't find any confirmation. Where is it?
Standalone question:
Can you help me locate my flight ticket confirmation?



> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer. Use only the chat history and the following information
Q: What's new in Altair SLC Version 4.4 A: Altair SLC version 4.4.1 onwards is available for purchase under Altair Units using Altair Licensing.

The Altair SLC 4.4 GA release contains new features and enhancements, and maintenance updates to existing functionality. Here are just a few headline items.

SAS Language Support Support for the ODS POWERPOINT output destination. Both the ODSLIST and ODSTEXT procedures now support the DATA option of the procedure statement, the CELLSTYLE statement and the TRANSLATE statement. The IMPORT procedure now supports the STARTCOL, ENDCOL, STARTROW, and ENDROW statements. The HTTP procedure now supports the DEBUG statement. The SGPANEL procedure now supports the TEXT statement. The SGPLOT procedure now supports the TEXT

QA Chain with Memory

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    input_key="question",
    output_key="answer",
    return_messages=True,
)

chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True
)

In [ ]:
question = "Tell about Altair SLC documentation"
docs = db.similarity_search(question)
answer = chain.run({"input_documents": docs, "question": question})
print(answer)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer. Use only the chat history and the following information
Q: What's new in Altair SLC Version 4.4 A: Altair SLC version 4.4.1 onwards is available for purchase under Altair Units using Altair Licensing.

The Altair SLC 4.4 GA release contains new features and enhancements, and maintenance updates to existing functionality. Here are just a few headline items.

SAS Language Support Support for the ODS POWERPOINT output destination. Both the ODSLIST and ODSTEXT procedures now support the DATA option of the procedure statement, the CELLSTYLE statement and the TRANSLATE statement. The IMPORT procedure now supports the STARTCOL, ENDCOL, STARTROW, and ENDROW statements. The HTTP procedure now supports the DEBUG statement. The SGPANEL procedure now supports the TEXT statement. The SGPLOT procedure now supports the TEXT, XAXISTABLE, and Y

KeyError: ignored